<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/kaggle/pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
file_url = 'https://raw.githubusercontent.com/krahul2024/small-datasets/main/melbourne-housing/melb_data.csv'
data = pd.read_csv(file_url)
data.head()

Preparing the test and training data

In [8]:
from sklearn.model_selection import train_test_split
house_data = data

# Target seperation , here it is price
y = house_data.Price
x = house_data.drop(['Price'], axis = 1)

# Data division into training and testing parts
x_train_total, x_valid_total, y_train, y_valid = train_test_split(
    x, y,
    test_size = 0.17,
    train_size = 0.83,
    random_state = 0
)

# Selecting categorical columns with low cardinality
object_cols = [
    col for col in x_train_total.columns
    if x_train_total[col].nunique() < 10 and x_train_total[col].dtypes == 'object'
]

# selecting numerical columns , here we don't have any problem with cardinality
num_cols = [
    col for col in x_train_total.columns
    if x_train_total[col].dtype in ['int64', 'float64']
]

# keeping selected columns, like categorical and numerical columns as we discard categorical columns with larger cardinality as of now
selected_cols = object_cols + num_cols
x_train = x_train_total[selected_cols].copy()
x_valid = x_valid_total[selected_cols].copy()


In [7]:
x_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


## Pipeline Construction

### 1. Pre-Processing Steps definition
*  Using ColumnTransformer Class to bundle different pre-processing steps together.
    * What we are doing is
        *  Imputation of missing values in * Numerical` data
        *  Imputation of missing values and one-hot encoding to categorical data

In [9]:
from sklearn.compose import ColumnTransformer  # for bundling
from sklearn.pipeline import Pipeline  # for creation of pipline
from sklearn.impute import SimpleImputer  # for imputation
from sklearn.preprocessing import OneHotEncoder  # for encoding categorical data

# preprocessing for numerical data
numerical_transformer = SimpleImputer(
    strategy = 'constant'
)

# Pre-processing for categorical data
categorical_transformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer( strategy = 'most_frequent')),
        ('onehot', OneHotEncoder( handle_unknown = 'ignore'))
    ]
)

# Bundling pre-processing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, object_cols)
    ]
)



### 2. Model Definition
* Define the model using RandomForestRegressor class

In [11]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 100, random_state = 0)

### 3. Creation and Evaluation of the  pipeline
* Using the pipeline class to define a pipeline which bundles the pre-processing the modelling steps.
    * Pre-process the training data with pipeline, and fit the model in a single line of code, without pipeline we would need to do a lot of stuff.
    * Supply unprocessed features in x_valid to the predict() function, and the pipeline automatically pre-processes the features before generating predictions.

In [12]:
from sklearn.metrics import mean_absolute_error

# bundle pre-processing and modeling code in pipeline
pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),  # for pre-processing
        ('model', model) # for creation of the model
    ]
)

pipe.fit(x_train, y_train)  # training or fitting the model

predictions = pipe.predict(x_valid)
mae = mean_absolute_error(
    y_valid, predictions
)
print('mean-absolute-error : ', mae)

mean-absolute-error :  157511.8280973004
